# ransac fitting ellipse

In [1]:
import numpy as np
import cv2
import math
import random

#Detect all valid dot centers in the frame
def find_mask(frame):
    blurred = cv2.blur(frame, (5, 5)) #Applies a blurring (smoothing) operation to reduce noise --> makes the detection more stable 
    mask = cv2.inRange(blurred, (0, 0, 0), (55, 95, 95))  # BGR Creates a binary mask by keeping only pixels within the BGR range (0, 0, 0) to (55, 95, 95). Output: A binary image (mask) where the detected dark pixels are white (255), and everything else is black (0).

    #Finds connected components in the binary mask.
    output = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
    num_labels, labels, stats = output[0], output[1], output[2]
    dots = []
    for i in range(1, num_labels):
        a = stats[i, cv2.CC_STAT_AREA]
        t = stats[i, cv2.CC_STAT_TOP]
        l = stats[i, cv2.CC_STAT_LEFT]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]

        if 5 < a < 400 and w/h < 3 and h/w < 3:
            dots.append((l + w//2, t + h//2))  #Computes and saves the center point of the blob.
            cv2.rectangle(frame, (l, t), (l + w, t + h), (12, 14, 230), 2) #Draws a bounding box around the blob in the original frame using a blue-purple color (12, 14, 230) and thickness 2.

    return mask, dots

# function estimates the parameters of an ellipse from a given set of points using a direct least-squares method (often used for conic sections like ellipses)
def fit_ellipse_direct(points):
    x = points[:, 0]  #all the x-coordinates
    y = points[:, 1]
    #builds the design matrix D for a general conic Each row in D corresponds to one point and represents the general conic form:ax^2+...=0
    D = np.vstack([x**2, x*y, y**2, x, y, np.ones_like(x)]).T
    
    #performs 'Singular Value 'Decomposition on the matrix D(vở xanh)
    _, _, V = np.linalg.svd(D) 
    params = V[-1, :]  #The last row of V gives the solution that minimizes the least squares problem  i.e., the best-fit ellipse.
    return params  # A, B, C, D, E, F

#This function computes the distance (residual--re'sid(tr)ual phần dư) of a single point (x,y) from the ellipse defined by params.
def ellipse_residual(params, point):
    x, y = point
    A, B, C, D, E, F = params
    val = A * x**2 + B * x * y + C * y**2 + D * x + E * y + F #These are the 6 parameters of the general conic e'quation. This equation des'cribes a conic section (which can be an ellipse, 'parabola, or 'hyperbola depending on values).
    return abs(val) #this value could be positive or negative --> absolute
'''
the result (val) shows how close this point is to lying on the ellipse: If val = 0, the point lies exactly on the curve. If val ≠ 0 — the larger the value, the further off it is.

Note: This is the 'algebraic distance, not 'geometric distance. It doesn’t 'measure "true" perpen'dicular distance, but it’s good enough for fast filtering in RANSAC.

'''

#To find the best-fitting ellipse from a set of 2D points, even when some of those points are outliers (noise or errors).
'''
Args:
        points (list of (x, y)): Detected dot centers.
        alpha (float): Expected inlier ratio.
        P (float): Desired probability of finding a good model.
        eps (float): Threshold to consider a point as an inlier.

    Returns:
        best_model (np.ndarray or None): Best ellipse parameters (A, B, C, D, E, F), or None if not found.

'''
def ransac_ellipse(points, alpha, P, eps):
    # Calculate required number of RANSAC iterations
    m = math.ceil(math.log2(1-P) / math.log2(1-alpha**5)) #iterations -- need 5 points to fit a model ellipse
    
    best_model = None
    max_inliers = 0
    n = len(points)
    if n < 5:  #We can’t fit an ellipse with fewer('fil .er) than 5 points
        return None

    for _ in range(m):
        sample_idx = random.sample(range(n), 5) # 'randomly sample 5 indexs, that correspond to 5 points are used to fit model 
        sample_points = np.array([points[i] for i in sample_idx])
        
        '''
        LinAlgError is an error raised by Python when you're performing a linear algebra operation that cannot be completed.
        The causes typically include:the points are: 1.Too close to each other 2.Collinear(lie in the same ) 3.Coincident(have at least two points are exactly the same) (overlapping)
        We use try-except to skip that iteration and select another set of 5 points
        '''
        try:
            params = fit_ellipse_direct(sample_points)
        except np.linalg.LinAlgError:
            continue

        inliers = 0
        for i in range(n):
            res = ellipse_residual(params, points[i])
            if res < eps:
                inliers += 1

        if inliers > max_inliers:
            max_inliers = inliers
            best_model = params

    return best_model

# draw the fitted ellipse use fitEllips --> The RANSAC algorithm helps find the best fitting ellipse model (a set of inlier points) from data that may contain many outliers, and then uses cv2.fitEllipse() to find the best ellipse for this set of inliers.
def draw_fitted_ellipse(frame, points):
    if len(points) < 5:
        return

    ellipse = cv2.fitEllipse(np.array(points))
    cv2.ellipse(frame, ellipse, (0, 255, 0), 2)


#filename = "dotswhiteboard.mp4"
filename = "dotswhitenoise.mp4"
cam = None

while True:
    if cam is None:
        reopen_cam = True
    else:
        success, frame = cam.read()
        if not success:
            cam.release()
            reopen_cam = True
        else:
            h, w, _ = frame.shape

    if reopen_cam:
        reopen_cam = False
        cam = cv2.VideoCapture(filename)
        continue
    frame = cv2.resize(frame, (600, 400))
    mask, dots = find_mask(frame)
    if len(dots) >= 5:
        ellipse_model = ransac_ellipse(dots, 0.6, 0.99, 15)
        if ellipse_model is not None:
            # Get inliers to draw a more accurate ellipse
            inliers = [pt for pt in dots if ellipse_residual(ellipse_model, pt) < 1e3]
            draw_fitted_ellipse(frame, inliers)

    cv2.imshow("frame", frame)
    cv2.imshow("mask", mask)

    key = cv2.waitKey(10) & 0xFF
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()
#cv2.waitKey(10)


# ransac fitting line

In [ ]:
import numpy as np
import cv2
import math
import time

#Finding Dots in Frame 
#reduce noise and them apply mask to Detect connected components (blobs) in the binary mask --> to get all center of each valid dot
def find_mask(frame):
    blurred=cv2.blur(frame, (5, 5))
    
    mask = cv2.inRange(blurred, (0, 0, 0), (50, 50, 50)) #BGR

    output = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)

    num_labels = output[0]
    labels = output[1]
    stats = output[2]

    dots = []

    for i in range(1, num_labels):
        a = stats[i, cv2.CC_STAT_AREA]
        t = stats[i, cv2.CC_STAT_TOP]
        l = stats[i, cv2.CC_STAT_LEFT]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        
        color = (123, 150, 12)
        
        #Filter blobs by size and shape. Save center point to dots.
        if (a > 10 and a < 400 and w/h <3 and h/w < 3):
            color = (12, 14, 230)
            dots.append((l + w//2,t+h//2))
    
        cv2.rectangle(frame, (l, t), (l+w, t+h), color, 3) # draw bounding box around the blob
    
    #dots[(400, 200), (300, 350), (100, 500))
    
    #cv2.imshow(":dsc", frame)
    return mask, dots  # Return the binary mask and list of detected dot centers.
    
'''
xmin and xmax define the range of x-values over which the line will be drawn
So you're drawing the line from point: (xmin, a*xmin + b) to (xmax, a*xmax + b)
'''
def draw_line(frame, a, b, xmin, xmax, color = (123, 23, 35)):

    y1 = int(xmin*a+b)
    y2= int(xmax*a+b)
    
    #print("ab", a, b)
    cv2. line(frame, (int(xmin), int(y1)), (int(xmax), int(y2)), color, 3)

#Draw line using polar system (r, θ) representation
def draw_line_r_theta(frame, r, theta, xmin, xmax, color = (123, 23, 35)):

    a = math.tan(theta)
    x0 = r*math.cos(theta + math.pi/2) 
    y0 = r*math.sin(theta + math.pi/2) 
    
    b = y0 - x0 * a
    
    draw_line(frame, a, b, xmin, xmax, color)

#Reformatting Dots--> Separates x and y coordinates from list of dots.-->Computes min and max x-values.
def repack_data(dots):
    n= len(dots)
    x= np.zeros((n))
    y= np.zeros((n))
    for i in range(n):
        x[i], y[i] = dots[i]
        
    xmin = min(x)
    xmax = max(x)
    
    return n, x, y, xmin, xmax

# perpen'dicularDistance from Point to Line
def calc_distance(a, b, dot):
    #Find the closest point (xp, yp) on the line to the input point (x, y)-->To do this, we project the point onto the line.
    x, y = dot
    xp = (y*a+x-b*a)/(a**2+1)
    yp = a*xp + b
    
    dist = math.sqrt((xp-x)**2+(yp-y)**2)

    return dist

#Convert (a, b) to polar representation(r, θ)
def r_theta_by_a_b(a, b): 
    r = b/math.sqrt(a**2+1)
    theta = math.atan(a)

    return r, theta

#RANSAC Line Fitting
def RANSAC(dots, alpha, P, eps, frame): 
    n, x, y, xmin, xmax = repack_data(dots)

    #RANSAC loop count m is computed from alpha (inlier ratio), P (confidence), and 2 points needed to define a line.
    m = math.ceil(math.log2(1-P) / math.log2(1-alpha**2))

    #Initialize best model and max inlier count.
    r_hat, theta_hat = 0, 0
    max_inl_num = 0

    
    for i in range(m):
        #subsample --> Randomly choose 2 points, calculate line y = ax + b
        p1 = dots[np.random.randint(n)]
        p2 = dots[np.random.randint(n)]

        #model
        #to handle two edge cases
        #p1 == p2:This checks if the two points are exactly the same.(You need two distinct points to define a line.)
        #p1[0] == p2[0]:This checks if the two points have the same x-coordinate, meaning the line is vertical.The slope a = (y2 - y1) / (x2 - x1) → here, you'd get division by zero if x2 == x1
        if (p1 == p2 or p1[0] == p2[0]):  
            continue
        
        a = (p1[1]-p2[1])/(p1[0] - p2[0]) 
        b = p1[1] - a*p1[0]
        
        #evaluate
        #Count how many points are within eps distance of the line
        inl_num = 0
        for j in range(n):
            curr_dist = calc_distance(a, b, dots[j])
        
            if (curr_dist < eps): 
                inl_num += 1

        if (max_inl_num < inl_num): 
            max_inl_num = inl_num

            r_hat, theta_hat = r_theta_by_a_b(a, b)

    return r_hat, theta_hat, xmin, xmax

#Main Video Processing Loop
filename = "dotswhiteboard.mp4"
#filename = "dotswhitenoise.mp4"

cam = None

while (True):
    if (cam is None):
        reopen_cam = True

    else:
        success, frame = cam.read()
        
        if (success == False): 
            cam.release()
            reopen_cam = True

        else:
            w, h, _ = frame.shape

    if (reopen_cam == True): 
        reopen_cam = False

        cam = cv2.VideoCapture(filename)

        continue
    frame = cv2.resize(frame, (600, 400))
    mask, dots = find_mask(frame)
    

    #RANSAC
    r, theta, xmin, xmax = RANSAC(dots, 0.6, 0.99, 15, frame) 
    draw_line_r_theta(frame, r, theta, xmin, xmax, (123, 234, 234))
    
    cv2.imshow("frame", frame)
    cv2.imshow("mask", mask)
    
    key = cv2.waitKey(100) & 0xFF
    
    if (key == ord('q')): 
        break

cam.release()
#cv2.waitKey(8)
cv2.destroyAllWindows()
cv2.waitKey(10)